In [1]:
import os 
import re
import matplotlib.pyplot as plt
import rmgpy.chemkin
import numpy as np
import cantera as ct
import random
import pandas as pd
%matplotlib inline

Let's get mapping of between RMG models and NIST

In [2]:
full_path_RMG = '/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/RMG_with_BROH'
chemkin_path_RMG = os.path.join(full_path_RMG,'chemkin/copies/copy_chem_annotated_204species.inp')
transport_path_RMG = os.path.join(full_path_RMG, 'chemkin/copies/tran.dat')
dictionary_path_RMG = os.path.join(full_path_RMG,'chemkin/species_dictionary.txt')

RMG_species, RMG_reactions = rmgpy.chemkin.load_chemkin_file(chemkin_path_RMG, dictionary_path=dictionary_path_RMG, transport_path=transport_path_RMG)

#NIST model 
full_path_NIST = '/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/NIST'
chemkin_path_NIST = os.path.join(full_path_NIST,'2-BTP_kinetics_with_M.inp')
dictionary_path_NIST = os.path.join(full_path_NIST,'species_dictionary-2_BTP.txt')

NIST_species, NIST_reactions = rmgpy.chemkin.load_chemkin_file(chemkin_path_NIST, dictionary_path=dictionary_path_NIST)


In [3]:
#model 1 = RMG, model 2 = NIST


In [11]:
#RMG has 1784 reactions (this is chemkin indexing, where each duplicate is counted)
chemkin_indices= list(range(1,1785))

1784

In [14]:
#let's flip each reaction one at a time

## Flipping 

In [3]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin
#import numpy as np
import subprocess
import csv
#import scipy
import copy
import os

In [ ]:
#functions 

In [4]:
def _2_to_1(_2_reaction):
    '''
    Convert the 2nd model species in the reactions to 1st model species, but keep the 2nd model kinetics
    '''
    
    _1_reaction = copy.deepcopy(_2_reaction)
    reactants = []
    for reactant in _2_reaction.reactants:
        try:
            _2_species_index = _2_species_list.index(reactant)
            reactants.append(_1_species_list[_2_to_1_mapping[_2_species_index]])
        except ValueError:
            if reactant in _1_species_list:
                reactants.append(reactant)
        
    _1_reaction.reactants = reactants
    
    products = []
    for product in _2_reaction.products:
        try:
            _2_species_index = _2_species_list.index(product)
            products.append(_1_species_list[_2_to_1_mapping[_2_species_index]])
        except ValueError:
            if product in _1_species_list:
                products.append(product)
    _1_reaction.products = products
    
    return _1_reaction

In [5]:
def _1_to_2(_1_reaction):
    # takes in the _1_reaction object to convert
    _1_index = _1_reaction_list.index(_1_reaction)
    if _1_index not in _1_to_2_rxn_mapping.keys():
        # this reaction does not exist in 2nd model, so it will be deleted. return None
        print(f'Reaction does not match to NIST reaction: {_1_reaction.index}')
        return
    _2_index = _1_to_2_rxn_mapping[_1_index]
    _2_reaction = _2_reaction_list[_2_index]
    
    # convert the 2nd model species in the 2nd model reaction to 1st model species
    return _2_to_1(_2_reaction)

In [6]:
#get mapping between 1st model and 2nd model 

#1st model is RMG, 2nd is NIST

#define the species 
_1_reaction_list = RMG_reactions
_1_species_list = RMG_species
_2_reaction_list = NIST_reactions
_2_species_list = NIST_species

# get the mapping between RMG and NIST models
# Species Diff

common_species = []
_1_to_2_mapping = {}
_2_to_1_mapping = {}
for i, _1_sp in enumerate(_1_species_list):
    for j, _2_sp in enumerate(_2_species_list):
        if _1_sp.is_isomorphic(_2_sp):
            _1_to_2_mapping[i] = j
            _2_to_1_mapping[j] = i
            common_species.append([_1_sp, _2_sp])
            break

# Reaction Diff
common_reactions = []
_1_to_2_rxn_mapping = {}
_2_to_1_rxn_mapping = {}
for i, _1_rxn in enumerate(_1_reaction_list):
    for j, _2_rxn in enumerate(_2_reaction_list):
        if _1_rxn.is_isomorphic(_2_rxn):
            _1_to_2_rxn_mapping[i] = j
            _2_to_1_rxn_mapping[j] = i
            common_reactions.append([_1_rxn, _2_rxn])
            break
print(f'{len(common_species)} common species')
print(f'{len(common_reactions)} common reactions')

57 common species
324 common reactions


In [9]:
#flipping (aka removing since there is no NIST equivalent) reactions CH02(230)

s = [1233, 561]
y = [1233, 561,624]
x = [561, 624]

chemkin_indices = [x]

In [10]:
############################### convert the indicated reactions to use NIST kinetics ##########################################
for rxns_with_CHO2 in chemkin_indices: 
    
    rxns_to_change = rxns_with_CHO2
    print(rxns_to_change)
    # convert the indicated reactions to use the NIST kinetics

    str_rxns_to_change = []
    for index in rxns_to_change: 
        str_rxns_to_change.append(str(index))

    file_name = '_'.join(str_rxns_to_change) 


    new_reaction_list = []
    deleted_duplicates = []



    for i in _1_reaction_list:
        if i.index in rxns_to_change:
            print(i.index)
            new_reaction = _1_to_2(i)
            if new_reaction:
                new_reaction_list.append(new_reaction)
            elif i.duplicate:
                deleted_duplicates.append(i)
        else:
            new_reaction_list.append(i)


    # get rid of duplicates
    for i, rxn in enumerate(new_reaction_list):
        if rxn.duplicate:
            duplicate_still_exists = False
            for j, rxn2 in enumerate(new_reaction_list):
                if rxn.is_isomorphic(rxn2) and rxn != rxn2:
                    duplicate_still_exists = True
                    break
            if not duplicate_still_exists:
                rxn.duplicate = False

    # mark reactions that are duplicates
    for i, rxn in enumerate(new_reaction_list):
        if not rxn.duplicate:
            duplicate = False
            for j, rxn2 in enumerate(new_reaction_list):
                if rxn.is_isomorphic(rxn2) and rxn != rxn2:
                    duplicate = True
                    break
            if duplicate:
                rxn.duplicate = True

    #save the chemkin file 

    handpicked_dir = '/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/RMG_with_BROH_specific_species_removed'


    chemkin_file = os.path.join(handpicked_dir, f'{file_name}.inp')
    rmgpy.chemkin.save_chemkin_file(chemkin_file, _1_species_list, new_reaction_list, verbose=True, check_for_duplicates=True)


[561, 624]
561
624


In [62]:
path_ = '/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/edited_RMG_with_BROH/'
files = [file for file in os.listdir(path_) if re.search('edited_RMG_with_BROH_diff_tol[\S]+\.inp', file)]
files



['edited_RMG_with_BROH_diff_tol_0.015-0.0_1161.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_1161_117.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_1161_37.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_1161_37_117.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_1161_37_777.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_1161_37_777_117.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_1161_777.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_1161_777_117.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_117.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_37.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_37_117.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_37_777.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_37_777_117.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_777.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_777_117.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_997.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_997_1161.inp',
 'edited_RMG_with_BROH_diff_tol_0.015-0.0_9

In [51]:
 list_ = ['copy_edited_RMG_with_BROH_difference_of_sensitivity_tolerances_0.015-0.0.cti',
 'copy_edited_RMG_with_BROH_difference_of_sensitivity_tolerances_0.015-0.01.cti']

In [52]:
len(list_)

2

In [53]:
' '.join(list_)

'copy_edited_RMG_with_BROH_difference_of_sensitivity_tolerances_0.015-0.0.cti copy_edited_RMG_with_BROH_difference_of_sensitivity_tolerances_0.015-0.01.cti'